In [1]:
import pandas as pd

# (Improper) Functions Import

In [2]:
from sklearn.preprocessing import StandardScaler

def std_scaler(df):
    """
    Scale the data with SKlearn StandardScaler
    """
    
    scaler = StandardScaler()
    
    col_to_scale_df = df.drop(columns=['ticker', 'date', '5TD_return', '10TD_return', '20TD_return'])
    col_to_scale = list(col_to_scale_df)
    
    scaled_df = df
    
    for col in col_to_scale:
        scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    
    return scaled_df

In [5]:
def build_input_arrays(df, time_window=5, stride=3, input_cols=['RSI', 'Stochastic', 'Stochastic_signal', 'ADI',
       'OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff',
       'MACD_signal'] , target_col='5TD_return'):
    output_array = []
    target_array = []
    df_sorted = df.sort_values('date', ascending=False)
    df_sorted.reset_index(drop=True, inplace=True)
    for row in range(0, len(df), stride):
        df_slice = df_sorted.iloc[row: row + time_window]
        if df_slice.shape[0]==time_window:
            output_array.append(np.array(df_slice[input_cols].values))
            target_array.append(df_slice[target_col].iloc[0])
    return np.array(output_array), np.array(target_array)

In [8]:
def train_test_split(df, test_set_size='3Y'):
    """
    Split the preprocessed stock data file into a train and test dataset
    INPUT: the dataframe to be split, and size of the test set in months or years ('3M' or '2Y')
    OUTPUT: returns a train_set and test_set dataframe, index is set to the date
    
    EXAMPLE: train_set, test_set = train_test_split(input_df, '3Y')
    """
    if not np.issubdtype(df['date'].dtype, np.datetime64):
        df['date'] = pd.to_datetime(df['date'], format=('%Y-%m-%d'))
    test_set = df.sort_values(by="date",ascending=True).set_index("date").last(test_set_size)
    train_set = df.drop(df.tail(len(test_set)).index).set_index("date")
    test_set.reset_index(inplace=True)
    train_set.reset_index(inplace=True)
    return train_set, test_set

# LSTM Baseline Model

In [3]:
df = pd.read_csv('../raw_data/AAPL.csv')

## Preprocessing

In [4]:
df_scaled = std_scaler(df)

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [6]:
df_scaled.head()

,ticker,date,RSI,Stochastic,Stochastic_signal,ADI,OBV,ATR,ADX,ADX_pos,ADX_neg,MACD,MACD_diff,MACD_signal,5TD_return,10TD_return,20TD_return
0,AAPL,1984-10-24,0.339732,0.691188,0.486039,-0.187390,-1.599369,-0.430690,-0.734613,1.133141,0.056752,-0.167518,0.000989,-0.176569,-0.052010,-0.018263,-0.119206
1,AAPL,1984-10-25,-0.328390,0.037753,0.447752,-0.191682,-1.601392,-0.430637,-0.827636,0.816135,0.775435,-0.167443,-0.000470,-0.176080,-0.009378,-0.022156,-0.060181
2,AAPL,1984-10-26,-0.692953,-0.393761,0.119486,-0.194098,-1.602861,-0.430709,-0.918523,0.596235,1.273968,-0.167725,-0.002715,-0.175749,0.009726,-0.055397,-0.026218
3,AAPL,1984-10-29,-0.639442,-0.340923,-0.248323,-0.195472,-1.602213,-0.431008,-1.002919,0.523564,1.191167,-0.167893,-0.004164,-0.175518,0.000000,-0.025925,-0.003162
4,AAPL,1984-10-30,-0.416737,-0.122524,-0.305440,-0.197496,-1.601259,-0.431131,-1.103085,0.758533,0.997380,-0.167834,-0.004643,-0.175322,0.048060,-0.060959,0.035265


In [12]:
train, test = train_test_split(df_scaled)

In [10]:
X

array([[[ 9.89523280e-01,  6.63277112e-01,  8.98370666e-01, ...,
          5.51259961e+00,  4.35071182e+00,  4.58129914e+00],
        [ 1.19131679e+00,  8.73454627e-01,  8.24127201e-01, ...,
          5.23010786e+00,  4.38009277e+00,  4.27577337e+00],
        [ 7.86643794e-01,  9.84083935e-01,  9.49720371e-01, ...,
          4.62318619e+00,  3.19927115e+00,  3.96818787e+00],
        [ 5.82762008e-01,  4.54941908e-01,  1.06237380e+00, ...,
          4.44482730e+00,  3.33159047e+00,  3.74338307e+00],
        [ 1.03870283e+00,  1.22588266e+00,  1.41526189e+00, ...,
          4.43746783e+00,  4.13873051e+00,  3.50930211e+00]],

       [[ 5.82762008e-01,  4.54941908e-01,  1.06237380e+00, ...,
          4.44482730e+00,  3.33159047e+00,  3.74338307e+00],
        [ 1.03870283e+00,  1.22588266e+00,  1.41526189e+00, ...,
          4.43746783e+00,  4.13873051e+00,  3.50930211e+00],
        [ 9.35548736e-01,  1.30020166e+00,  1.13300132e+00, ...,
          3.98798092e+00,  3.48840272e+00,  3.21863

In [11]:
y

array([ 0.01321014,  0.07923098,  0.03127932, ...,  0.        ,
       -0.10401985, -0.0128992 ])

In [14]:
X_train, y_train = build_input_arrays(train)
X_test, y_test = build_input_arrays(test)

In [15]:
X_train

array([[[-7.86138104e-01, -1.63389050e+00, -1.29998695e+00, ...,
          2.02412752e-02, -1.01746934e+00,  3.06617396e-01],
        [-4.00207826e-01, -9.10378599e-01, -9.15620724e-01, ...,
          1.55769389e-01, -7.61353740e-01,  3.77424352e-01],
        [-5.18041835e-01, -1.10374481e+00, -3.09607546e-01, ...,
          2.42138554e-01, -6.25697061e-01,  4.30276496e-01],
        [-5.24625257e-01, -5.55317224e-01,  3.65314663e-01, ...,
          3.68324395e-01, -3.06698858e-01,  4.73618521e-01],
        [ 5.36249699e-01,  7.90160010e-01,  7.74759605e-01, ...,
          5.22877798e-01,  1.98509057e-01,  4.94597391e-01]],

       [[-5.24625257e-01, -5.55317224e-01,  3.65314663e-01, ...,
          3.68324395e-01, -3.06698858e-01,  4.73618521e-01],
        [ 5.36249699e-01,  7.90160010e-01,  7.74759605e-01, ...,
          5.22877798e-01,  1.98509057e-01,  4.94597391e-01],
        [ 5.36249699e-01,  7.90160010e-01,  7.22681082e-01, ...,
          5.15249926e-01,  2.21371483e-01,  4.80159

In [16]:
y_train

array([ 0.0305561 ,  0.00974801, -0.01871454, ..., -0.03888713,
       -0.02592475,  0.03559837])

In [17]:
X_test

array([[[ 9.89523280e-01,  6.63277112e-01,  8.98370666e-01, ...,
          5.51259961e+00,  4.35071182e+00,  4.58129914e+00],
        [ 1.19131679e+00,  8.73454627e-01,  8.24127201e-01, ...,
          5.23010786e+00,  4.38009277e+00,  4.27577337e+00],
        [ 7.86643794e-01,  9.84083935e-01,  9.49720371e-01, ...,
          4.62318619e+00,  3.19927115e+00,  3.96818787e+00],
        [ 5.82762008e-01,  4.54941908e-01,  1.06237380e+00, ...,
          4.44482730e+00,  3.33159047e+00,  3.74338307e+00],
        [ 1.03870283e+00,  1.22588266e+00,  1.41526189e+00, ...,
          4.43746783e+00,  4.13873051e+00,  3.50930211e+00]],

       [[ 5.82762008e-01,  4.54941908e-01,  1.06237380e+00, ...,
          4.44482730e+00,  3.33159047e+00,  3.74338307e+00],
        [ 1.03870283e+00,  1.22588266e+00,  1.41526189e+00, ...,
          4.43746783e+00,  4.13873051e+00,  3.50930211e+00],
        [ 9.35548736e-01,  1.30020166e+00,  1.13300132e+00, ...,
          3.98798092e+00,  3.48840272e+00,  3.21863

In [18]:
y_test

array([ 0.01321014,  0.07923098,  0.03127932,  0.0443038 , -0.01591919,
       -0.01056224,  0.04854619,  0.07790953, -0.01694484, -0.01609928,
        0.02949039,  0.00328228,  0.07128385,  0.03369066, -0.05372692,
       -0.00775722, -0.04107884, -0.06771882,  0.05309889,  0.03491979,
       -0.00807797,  0.07923725,  0.04431704, -0.04466679, -0.04606028,
        0.02406108, -0.13631166, -0.03089888,  0.07495386,  0.05682588,
        0.09815576,  0.02384329,  0.03416877, -0.00082357,  0.18410483,
        0.14895689, -0.02294416, -0.03854608, -0.00058936,  0.00878904,
        0.0215496 ,  0.04542353, -0.00200246,  0.00812071,  0.02406695,
        0.03222672,  0.02353724,  0.04215652,  0.03606885,  0.02200686,
        0.00343325,  0.00562048,  0.03764205, -0.00777903,  0.04934397,
        0.033811  ,  0.03525031,  0.04214654,  0.000612  , -0.06506163,
        0.07752935,  0.13186282,  0.0202292 , -0.05226682,  0.13565586,
       -0.00463416, -0.17530878, -0.11381942, -0.15255448, -0.10

## Model Initialisation

## Model Evaluation

# Model Improvement